# Database Operations and File Handling

This notebook covers the steps to create and manipulate database tables, as well as copying files to specific directories.


## 1. Create the `stg_Churn` Table


DROP TABLE IF EXISTS stg_Churn;

CREATE TABLE stg_Churn (
    Customer_ID VARCHAR(20),
    Gender VARCHAR(10),
    Age INT,
    Married VARCHAR(5),
    State VARCHAR(50),
    Number_of_Referrals INT,
    Tenure_in_Months INT,
    Value_Deal VARCHAR(50),
    Phone_Service VARCHAR(5),
    Multiple_Lines VARCHAR(5),
    Internet_Service VARCHAR(5),
    Internet_Type VARCHAR(50),
    Online_Security VARCHAR(5),
    Online_Backup VARCHAR(5),
    Device_Protection_Plan VARCHAR(5),
    Premium_Support VARCHAR(5),
    Streaming_TV VARCHAR(5),
    Streaming_Movies VARCHAR(5),
    Streaming_Music VARCHAR(5),
    Unlimited_Data VARCHAR(5),
    Contract VARCHAR(20),
    Paperless_Billing VARCHAR(5),
    Payment_Method VARCHAR(20),
    Monthly_Charge NUMERIC,
    Total_Charges NUMERIC,
    Total_Refunds NUMERIC,
    Total_Extra_Data_Charges NUMERIC,
    Total_Long_Distance_Charges NUMERIC,
    Total_Revenue NUMERIC,
    Customer_Status VARCHAR(20),
    Churn_Category VARCHAR(50),
    Churn_Reason TEXT
);

## 2. Import the Table Data


COPY stg_Churn FROM '/Users/yiluo/Desktop/project-aish/Data & Resources/Data/Customer_Data.csv'
DELIMITER ','
CSV HEADER;

## 3. Explore the Database Using SQL


SELECT 
    gender, 
    COUNT(gender) AS TotalCount,
    COUNT(gender) * 1.0 / (SELECT COUNT(*) FROM stg_churn) AS Percentage
FROM 
    stg_churn
GROUP BY 
    gender;

## 4. Create the `prod_Churn` Table


CREATE TABLE prod_Churn AS
SELECT 
    Customer_ID,
    Gender,
    Age,
    Married,
    State,
    Number_of_Referrals,
    Tenure_in_Months,
    COALESCE(Value_Deal, 'None') AS Value_Deal,
    Phone_Service,
    COALESCE(Multiple_Lines, 'No') AS Multiple_Lines,
    Internet_Service,
    COALESCE(Internet_Type, 'None') AS Internet_Type,
    COALESCE(Online_Security, 'No') AS Online_Security,
    COALESCE(Online_Backup, 'No') AS Online_Backup,
    COALESCE(Device_Protection_Plan, 'No') AS Device_Protection_Plan,
    COALESCE(Premium_Support, 'No') AS Premium_Support,
    COALESCE(Streaming_TV, 'No') AS Streaming_TV,
    COALESCE(Streaming_Movies, 'No') AS Streaming_Movies,
    COALESCE(Streaming_Music, 'No') AS Streaming_Music,
    COALESCE(Unlimited_Data, 'No') AS Unlimited_Data,
    Contract,
    Paperless_Billing,
    Payment_Method,
    Monthly_Charge,
    Total_Charges,
    Total_Refunds,
    Total_Extra_Data_Charges,
    Total_Long_Distance_Charges,
    Total_Revenue,
    Customer_Status,
    COALESCE(Churn_Category, 'Others') AS Churn_Category,
    COALESCE(Churn_Reason , 'Others') AS Churn_Reason
FROM stg_Churn;

## 5. Create Views


CREATE VIEW vw_ChurnData AS
SELECT *
FROM prod_Churn
WHERE Customer_Status IN ('Churned', 'Stayed');

CREATE VIEW vw_JoinData AS
SELECT *
FROM prod_Churn
WHERE Customer_Status = 'Joined';

## 6. Save the Driver File in the Required Path


cp /Users/yiluo/Downloads/postgresql-42.7.3.jar ~/Library/Tableau/Drivers/

## 7. Data Transformation in Tableau

1. Connect the Postgresql database in tableau
2. Then select `prod_churn` table.
3. At bottom left side, click on Sheet 1 to see the Sheet View of the data.
4. Click on the dropdown beside the table name to see the Menu.
5. Select 'Create Calculated Field' under the Menu.
to create first field 'Churn Status' --> `IF [Customer Status] = "Churned" THEN 1 ELSE 0 END`
6. Second field "Monthly Charge Range' --> `IF [Monthly Charge] < 20 THEN "< 20"
ELSEIF [Monthly Charge] < 50 THEN "20-50"
ELSEIF [Monthly Charge] < 100 THEN "50-100"
ELSE "> 100" END`
7. Go to Data Source tab view, click on the dropdown beside Age column , then click on 'Create Calculated Field' to set the 'Age Group' --> `IF [Age] < 20 THEN "< 20"
ELSEIF [Age] < 36 THEN "20 - 35"
ELSEIF [Age] < 51 THEN "36 - 50"
ELSE "> 50" END`
8. Now move to Sheet View, click the dropdown select 'Create Calculated Field' to sort the age group `AgeGrpSorting` --> `IF [Age Group] = "< 20" THEN 1
ELSEIF [Age Group] = "20 - 35" THEN 2
ELSEIF [Age Group] = "36 - 50" THEN 3
ELSE 4 END`
check whether this field is numeric or not by right click on it --> change data type --> it should be Number (whole)
9. Go to Data Source tab view, click on the dropdown beside Tenure In Months column , then click on 'Create Calculated Field' to set the 'Tenure Group' --> `IF [Tenure In Months] < 6 THEN "< 6 Months"
ELSEIF [Tenure In Months] < 12 THEN "6-12 Months"
ELSEIF [Tenure In Months] < 18 THEN "12-18 Months"
ELSEIF [Tenure In Months] < 24 THEN "18-24 Months"
ELSE ">= 24 Months" END`
10. Now move to Sheet View, click the dropdown select 'Create Calculated Field' to sort the age group `TenureGrpSorting` --> `IF [Tenure Group] = "< 6 Months" THEN 1
ELSEIF [Tenure Group] = "6-12 Months" THEN 2
ELSEIF [Tenure Group] = "12-18 Months" THEN 3
ELSEIF [Tenure Group] = "18-24 Months" THEN 4
ELSE 5 END`
check whether this field is numeric or not by right click on it --> change data type --> it should be Number (whole)
11. Total Customers --> `COUNT([Customer Id])`
12. New Joiners --> `COUNT(IF [Customer Status] = "Joined" THEN [Customer Id] END)`
13. Total Churn --> `SUM([Churn Status])`
14. Churn Rate --> `[Total Churn] / [Total Customers]`
